In [90]:
from keras.models import Model
from keras import backend as K
import numpy as np
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [78]:
os.chdir("C:/Users/simas/OneDrive/Desktop/alawais midi")

In [79]:
# Load your encoded MIDI sequences from a .npy file
encoded_sequences = np.load("C:/Users/simas/OneDrive/Desktop/alawais midi/encoded_sequences.npy")

In [101]:
    # Define the dimensions of your encoded sequence and the bottleneck layer
encoding_dim = 64
input_shape = encoded_sequences.shape[1:]

In [102]:
# Define the input layer with Flatten layer
input_layer = Input(shape=input_shape)
flatten_layer = Flatten()(input_layer)


In [103]:
# Define the encoder layers
encoded = Dense(64, activation='relu')(flatten_layer)
encoded = Dense(32, activation='relu')(encoded)


# Define the bottleneck layer
z_mean = Dense(encoding_dim)(encoded)
z_log_var = Dense(encoding_dim)(encoded)


In [104]:
# Define the sampling function for the VAE
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], encoding_dim),
                              mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


In [105]:
# Use the sampling function to sample from the learned distribution
z = Lambda(sampling)([z_mean, z_log_var])


In [106]:

# Define the decoder layers
decoded = Dense(32, activation='relu')(z)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(np.prod(input_shape), activation='sigmoid')(decoded)
decoded = Reshape(input_shape)(decoded)


In [107]:

# Define the VAE model
vae = Model(inputs=input_layer, outputs=decoded)

In [108]:
# Define the loss function for the VAE
reconstruction_loss = K.binary_crossentropy(input_layer, decoded)
kl_loss = -0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)


In [109]:

# Compile the VAE with Adam optimizer and binary crossentropy loss
vae.compile(optimizer='adam', loss='binary_crossentropy')

In [110]:
# Split your dataset into a training set and validation set
train_data, val_data = train_test_split(encoded_sequences, test_size=0.2)

In [111]:
print(input_layer.shape)
print(decoded.shape)
print(train_data.shape)

(None, 32, 3, 128)
(None, 32, 3, 128)
(60203, 32, 3, 128)


In [100]:
vae.fit(train_data, train_data, epochs=50, batch_size=128, shuffle=True, validation_data=(val_data, None))


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/model_7/tf.__operators__.add_11/BroadcastGradientArgs' defined at (most recent call last):
    File "c:\Users\simas\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\simas\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\simas\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\simas\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\simas\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\simas\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\simas\anaconda3\lib\asyncio\base_events.py", line 